## The forward and backward passes

In [1]:
# !pip install torch fastcore

In [2]:
import torch

In [3]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

In [4]:
path_data = '../data'
path_gz = f'{path_data}/mnist.pkl.gz.1'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations version

### Basic architecture

In [5]:
y_train.shape

torch.Size([50000])

In [6]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [7]:
# num hidden
h = 50

In [8]:
w1 = torch.randn(m,h)
b1 = torch.zeros(h)
w2 = torch.randn(h,1)
b2 = torch.zeros(1)

In [9]:
def lin(x, w, b):
    """
    This function takes in n examples from a space of dimensionality m and 
    projects it down to a dimensionality of h.
    """
    # Arg Shapes      x: [n, m]   w: [m, h]   b: [h]
    # ...
    # Output Shapes   [n, h]
    return x @ w + b

In [10]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [11]:
def relu(x: torch.Tensor): return x.clamp_min(0.)

In [12]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [13]:
def model(xb):
    # Layer 1
    # Linear 
    l1 = lin(xb, w1, b1)
    # Non linear activation
    l2 = relu(l1)
    # Layer 2
    # Linear
    return lin(l2, w2, b2)

In [14]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [15]:
res.squeeze()

tensor([  25.75,  -13.06, -114.79,  ...,  -67.44,  -74.48,  -60.19])

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [16]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

Look at what happens if we don't remove the trailing dimension! Remember our broadcasting rules:
* We look at the trailing dimensions. We first compare the 10000 to the 1. They are compatible because the 1 will be broadcast ("copied") 10000 times, ensuring the dimensions match.
* We then move to the next dimension, 10000 for the first dimension of `res`. Now there is no further dimension of `y_valid`, so torch actually inserts a dimension for us - it inserts a unit axis of 1! 



In [17]:
(res-y_valid).shape

torch.Size([10000, 10000])

We need to get rid of that trailing (,1), in order to use `mse`.

In [18]:
res[:,0].shape

torch.Size([10000])

In [19]:
res.squeeze().shape

torch.Size([10000])

In [20]:
(res[:,0]-y_valid).shape

torch.Size([10000])

In [21]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [22]:
def mse(prediction, target): 
    return (prediction.squeeze() - target).pow(2).mean()

In [23]:
y_train.shape

torch.Size([50000])

In [24]:
mse(preds, y_train)

tensor(4308.76)

### Gradients and backward pass

In [25]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [26]:
diff(3*x**2+9, x)

6*x

In [27]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    # Note: We need the output so that we can use it for chain rule
    # This is effectively an outer product
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    # w.g = (inp * out.g).sum(0)
    b.g = out.g.sum(0)

In [28]:
def forward_and_backward(xb, target):
    
    # ------------- Forward Pass -------------
    # Layer 1
    l1 = lin(xb, w1, b1) 
    l2 = relu(l1)
    # Layer 2
    output = lin(l2, w2, b2)
    # Loss
    diff = target - output.squeeze() 
    loss = diff.pow(2).mean()
    
    # ------------- Backward Pass -------------
    # Gradient of the loss wrt the output
    output.g = -2. * diff[:, None] / xb.shape[0]

    # Gradient of loss wrt l2, w2 and b2
    lin_grad(l2, output, w2, b2)
    # Gradient of loss wrt l2, w2 and b2
    l1.g = (l1>0).float() * l2.g
    # Gradient of loss wrt xb, w1 and b1
    lin_grad(xb, l1, w1, b1)

In [32]:
l1.g = 'test'

NameError: name 'l1' is not defined

In [33]:
xb = x_train
yb = y_train

target = yb

l1 = lin(xb, w1, b1) 
l2 = relu(l1)
# Layer 2
output = lin(l2, w2, b2)
# Loss
diff = output.squeeze() - target
loss = diff.pow(2).mean()

In [34]:
output.g = -2. * diff[:, None] / xb.shape[0]

In [35]:
output.g.shape

torch.Size([50000, 1])

In [36]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    # Note: We need the output so that we can use it for chain rule
    # This is effectively an outer product
    inp.g = out.g @ w.t()
    w.g = (inp * out.g).sum(0)
    b.g = out.g.sum(0)

In [37]:
lin_grad(l2, output, w2, b2)

In [38]:
output.g.shape

torch.Size([50000, 1])

In [39]:
w2.shape

torch.Size([50, 1])

In [40]:
l2.g.shape

torch.Size([50000, 50])

In [41]:
l2.g

tensor([[    -0.00,      0.00,      0.00,  ...,     -0.00,      0.00,     -0.00],
        [    -0.00,      0.00,      0.00,  ...,     -0.00,      0.01,     -0.01],
        [     0.00,     -0.00,     -0.00,  ...,      0.00,     -0.00,      0.00],
        ...,
        [    -0.00,      0.00,      0.00,  ...,     -0.00,      0.00,     -0.00],
        [    -0.00,      0.00,      0.00,  ...,     -0.00,      0.00,     -0.00],
        [    -0.00,      0.00,      0.00,  ...,     -0.00,      0.00,     -0.00]])

In [42]:
# Think of l2 as out input "x". It is a batch of 50000 examples, of dim 50
l2.unsqueeze(-1).shape 

torch.Size([50000, 50, 1])

In [43]:
# It is then transformed via the affine function wx + b and the output dimension is 1
output.g.unsqueeze(1).shape

torch.Size([50000, 1, 1])

In [44]:
(l2 * output.g).shape

torch.Size([50000, 50])

In [45]:
(l2 * output.g).sum(dim=0)

tensor([ 869.79,  622.49,  165.30,  468.34,  837.93,  687.30,  559.73,  637.38,  176.24,   10.81,  993.01,  533.38,  730.86,
          54.01,  451.91,  156.28,  567.42,   61.53,   15.62,  463.00,   79.85,  582.16,   11.21, 1023.28,  140.96, 1357.97,
        1326.60,  115.92,  125.99,    4.39,  601.42,  508.06,  505.85,  301.20,  549.87,   13.01,  900.70,  179.80,  117.92,
        1531.71,  295.88,  327.67,  642.84,    7.32, 1333.88,   10.38,  198.15,  561.56,   53.83, 1659.88])

In [46]:
(l2.unsqueeze(-1) * output.g.unsqueeze(1)).sum(dim=0)

tensor([[ 869.79],
        [ 622.49],
        [ 165.30],
        [ 468.34],
        [ 837.93],
        [ 687.30],
        [ 559.73],
        [ 637.38],
        [ 176.24],
        [  10.81],
        [ 993.01],
        [ 533.38],
        [ 730.86],
        [  54.01],
        [ 451.91],
        [ 156.28],
        [ 567.42],
        [  61.53],
        [  15.62],
        [ 463.00],
        [  79.85],
        [ 582.16],
        [  11.21],
        [1023.28],
        [ 140.96],
        [1357.97],
        [1326.60],
        [ 115.92],
        [ 125.99],
        [   4.39],
        [ 601.42],
        [ 508.06],
        [ 505.85],
        [ 301.20],
        [ 549.87],
        [  13.01],
        [ 900.70],
        [ 179.80],
        [ 117.92],
        [1531.71],
        [ 295.88],
        [ 327.67],
        [ 642.84],
        [   7.32],
        [1333.88],
        [  10.38],
        [ 198.15],
        [ 561.56],
        [  53.83],
        [1659.88]])

In [47]:
(output.g @ w2.T).shape

torch.Size([50000, 50])

In [48]:
forward_and_backward(x_train, y_train)

RuntimeError: The size of tensor a (784) must match the size of tensor b (50) at non-singleton dimension 1

In [49]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

AttributeError: 'Tensor' object has no attribute 'g'

We cheat a little bit and use PyTorch autograd to check our results.

In [50]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [51]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [52]:
loss = forward(xt2, y_train)
loss.backward()

In [53]:
xt2.grad

tensor([[    -0.00,     -0.01,      0.00,  ...,     -0.00,      0.00,      0.00],
        [    -0.03,     -0.03,      0.01,  ...,     -0.04,     -0.01,     -0.01],
        [     0.00,      0.00,     -0.00,  ...,      0.00,     -0.00,      0.00],
        ...,
        [    -0.00,     -0.02,      0.01,  ...,     -0.00,     -0.00,      0.00],
        [    -0.02,     -0.01,      0.01,  ...,     -0.01,      0.01,     -0.00],
        [    -0.00,     -0.00,      0.00,  ...,     -0.00,     -0.00,     -0.00]])

In [44]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

## Refactor model

### Layers as classes

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [ ]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [ ]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])